In [ ]:
# All important libraries goes here!
import pandas as pd

In [17]:
dataframe = pd.read_csv('./Data/washedData.csv')
dataframe.head(300)

,ID,No of employee,Annual turnover,TCTC,Basic Salary,Cash Injection,Contrib Waiver,Affected Employee
0,1,63,3098000000,13782989.04,9500520.21,0,1,0
1,2,73,15000000,326574.61,992400,1,1,1
2,3,18,0,543629.21,397789.63,0,1,1
3,4,25,52000000,725607.67,496910,0,1,1
4,5,1,765270.6,205385.34,31530,0,0,0
...,...,...,...,...,...,...,...,...
295,296,1,108,99,1500,0,1,0
296,297,1,/A,/A,1800,1,1,1
297,298,3,40000,0,0,1,1,1
298,299,3,0,0,500,1,1,1


#### Problem Statement
the problem explaining the purpose of collecting this data, what
problem it is intended to solve,

Why are we using this data set and what problem are
you trying to solve.


<br>
<br>

#### Data cleaning

* Handling duplicate values.
* Dealing with missing/null values.
* Addressing inconsistent data.

We will go through every column

In [ ]:
# Code goes here!

<br>
<br>

#### Statistical Analysis

* Mean TCTC: Explain the average compensation.
* Median Basic Salary: Discuss the central tendency.
* Standard Deviation of Annual Turnover: Highlight variability.

In [ ]:
# Code goes here!

<br>
<br>

#### Exploratory Data Analysis (EDA)

* Histograms of "Annual turnover" and "Basic Salary."
* Box plots of "TCTC" to identify outliers.
* Scatter plots to explore relationships between variables.

Interpret the insights gained from each visualization.

In [ ]:
# Code goes here!
# Ever exploration is to be done on an individual block.
# With a markdown to explain the Exploration

<br>
<br>

#### Machine Learning

##### Predictive Modeling for Annual Turnover

Using **Annual Turnover** as the target variable (what you want to predict) and use other features as predictors.

* No of Employees
* TCTC
* Basic Salary
* Cash Injection
* Contrib Waiver

we can use regression algorithms like linear regression, decision trees, or random forests to build the model.

<br>

##### Employee Classification

We can use machine learning to classify employees into different categories based

* Cash Injection
* Contrib Waiver
* Affected Employee

We might want to classify employees into **Highly Affected** and **Less Affected** categories.
We can use classification algorithms like logistic regression, decision trees, or support vector machines.

<br>

##### Employee Segmentation

Clustering techniques like K-means clustering can be used to segment employees based on their characteristics.

We can use features like:

* No of Employees
* TCTC
* Basic Salary

To create meaningful clusters

In [ ]:
# Code goes here!

<br>
<br>

#### Evaluation of Machine Learning

Present performance metrics (e.g., Mean Absolute Error, R-squared) for each algorithm used.

Explain what the results mean:

* Which algorithm performed better?
* How accurate is the prediction of turnover?

<br>
<br>

#### Presentation of Results

Summarize key findings:

* Trends in employee turnover.
* Compensation fairness insights.

Mention any actionable recommendations based on the analysis.